To Do:
- Make List page have subpages (e.g., list of forms, list of forms using a field, see below), make these fileres visible after the intro paragraph
- Make list of forms using a given field with description with embeded question bank info

Thought: make a pdf editor the first step in the weaver. Leverage the name normalizer but allow folks to add and remove fields and rename. 

In [1]:
import pandas as pd
import re
#import PyPDF2
import os
from os import walk
import os.path
from os import path
import shutil

##import time
from datetime import date

from joblib import dump, load

import urllib.parse

In [2]:
import formfyxer as lit

In [3]:
jurs = [
        ["AK","Alaska"],
        ["AL","Alabama"],
        ["CA","California"],
        ["CO","Colorado"],
        ["CT","Connecticut"],
        ["DE","Delaware"],
        ["HI","Hawaii"],
        ["ID","Idaho"],
        ["IL","Illinois"],
        ["KS","Kansas"],
        ["MA","Massachusetts"],
        ["ME","Maine"],
        ["MD","Maryland"],
        ["MI","Michigan"],
        ["MN","Minnesota"],
        ["NC","North Carolina"],
        ["NE","Nebraska"],
        ["NV","Nevada"],
        ["SD","South Dakota"],
        ["UT","Utah"],
        ["VA","Virginia"],
        ["VT","Vermont"],
        ["WA","Washington"],
        ["WI","Wisconsin"],
        ["WY","Wyoming"],
        ]

default_jur = "MA"

today = date.today().strftime("%Y-%m-%d")

In [4]:
def removeSpecial(text):
    text = re.sub('[^a-zA-Z0-9]',"_",text)
    return re.sub('_+',"_",text)

In [5]:
def clean_n(n):
    if pd.notnull(n):
        return int(n)
    else:
        return 0

In [6]:
included_fields = [
                    "users1_name",
                    "users1_birthdate",
                    "users1_address_line_one",
                    "users1_address_line_two",
                    "users1_address_city",
                    "users1_address_state",
                    "users1_address_zip",
                    "users1_phone_number",
                    "users1_email",
                    "plantiffs1_name",
                    "defendants1_name",
                    "petitioners1_name",
                    "respondents1_name",
                    "docket_number",
                    "trial_court_county",
                    "users1_signature",
                    "signature_date"
                    ]

files_df = pd.read_csv("../data/processed/form_data.csv")

print(len(files_df))
display(files_df.head())

files_sim_df = pd.read_csv("../data/processed/form_sim_data.csv")

print(len(files_sim_df))
files_sim_df.head()

10018


,id,jurisdiction,source,title,group,url,filename,downloaded,pages,fields,fields_conf,fields_old,f_per_p,reading,list,text,downloaded_on
0,04b3a0734774c02edf8eb9056d23954aa38e96c77c3392...,UT,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-11,2.0,"['name__1', 'name__2', '*docket_number', '*use...","[0.63, 0.61, 1, 1, 1, 0.61, 0.52, 0.53, 0.6, 0...","['name__1', 'name__2', 'docket_number', 'users...",7.500000,10.5,[],COMMUNITY SERVICE WORKSHEET\n THIRD DISTRICT J...,NaN
1,6e420f1b3575cfd8ef94b71977da9e38252e3395a78439...,UT,www.utcourts.gov,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Work_Crew_Application-2007.pdf,2021-11-11,2.0,"['*docket_number', '*users1_birthdate', 'male'...","[1, 1, 0.69, 0.61, 1, 1, 1, 0.56, 0.68, 0.67, ...","['docket_number', 'users1_birthdate', 'male', ...",19.000000,10.5,['GO-00-00-00-00'],THIRD DISTRICT JUVENILE COURT WORK PROGRAM REF...,NaN
2,2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065...,UT,www.utcourts.gov,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Probation%20Order%20revised.050502.pdf,2021-11-11,5.0,"['probation_department_order_court', 'special_...","[0.73, 0.65, 0.7, 0.54, 0.68, 0.67, 0.62, 0.59...",['of the probation department or by order of t...,4.800000,12.5,[],Page 1 of 4\n UTAH STATE DISTRICT JUVENILE COU...,NaN
3,f6a6814890f21c11524d5785d772272916ff95909b7dba...,UT,www.utcourts.gov,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,02_Consent_Adoptee.pdf,2021-11-11,3.0,"['*users1_name', '*users1_address_line_one', '...","[1.0, 1.0, 1.0, 1.0, 0.61, 0.7, 0.67, 1.0, 0.6...","['Name', 'Address', 'City State Zip', 'Phone',...",14.666667,6.5,[],Consent to Adoption and Waiver of \n Rights\n ...,NaN
4,be656a9a361db7c3532026b0bae372e704beb160fce37e...,UT,www.utcourts.gov,Adoption Agreement,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,08_Agreement.pdf,2021-11-11,1.0,"['*users1_name', '*users1_address_line_one', '...","[1.0, 1.0, 1.0, 1.0, 0.61, 0.7, 0.67, 1.0, 0.6...","['Name', 'Address', 'City State Zip', 'Phone',...",19.000000,7.5,['GO-00-00-00-00'],Adoption\n \n Agreement\n \n Approved Board of...,NaN


3162622


,id_1,id_2,title_2,similarity
0,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,eb4e35c3b9f355d63ad14ae71866d44b14e61196763553...,Answer to Complaint to Recover Possession of P...,0.979167
1,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,74723a905136a3e30a9eec9aa9191f5d3e4b800ae319df...,"Answer, Damage/Health Hazard to Property, Land...",0.937500
2,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,ee10d4fd30c1faf309bd7b1a91da0794ec8862ee91c355...,"Answer, Damage/Health Hazard to Property, Land...",0.918367
3,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,5345e31f71a6ac7dc730e96039d35c9fc5d2cda658332e...,"Answer, Nonpayment of Rent, Landlord-Tenant",0.918367
4,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,98d5e7eedefb48b57a67dcbf928893c8032c1dec85e867...,"Answer, Nonpayment of Rent, Landlord-Tenant",0.937500


In [7]:
csv_df = files_df.copy()
csv_df["new_url"] = "https://courtformsonline.org/forms/"+csv_df["id"]+".pdf"
csv_df.loc[pd.isnull(csv_df["downloaded_on"]),"downloaded_on"] = csv_df["downloaded"]
csv_df = csv_df.rename(columns={"list":"list_codes","reading":"reading_level","fields": "standardized_fields","url":"original_url","fields_old":"original_fields"})
for state in csv_df["jurisdiction"].unique():
    tmp_df = csv_df[csv_df["jurisdiction"]==state][["title","group","pages","reading_level","downloaded_on","original_url","original_fields","new_url","standardized_fields","jurisdiction"]]
    #display(tmp_df)
    tmp_df.to_csv("docs/data/%s_form_data.csv"%(state), index=False, encoding="utf-8")    

tmp_df = csv_df[["title","group","list_codes","pages","f_per_p","reading_level","downloaded_on","original_url","original_fields","new_url","standardized_fields","jurisdiction","text"]]
tmp_df.to_csv("docs/data/form_data.csv", index=False, encoding="utf-8")       
tmp_df.head()

,title,group,list_codes,pages,f_per_p,reading_level,downloaded_on,original_url,original_fields,new_url,standardized_fields,jurisdiction,text
0,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,[],2.0,7.500000,10.5,2021-11-11,https://www.utcourts.gov/courts/juv/juvsites/3...,"['name__1', 'name__2', 'docket_number', 'users...",https://courtformsonline.org/forms/04b3a073477...,"['name__1', 'name__2', '*docket_number', '*use...",UT,COMMUNITY SERVICE WORKSHEET\n THIRD DISTRICT J...
1,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,['GO-00-00-00-00'],2.0,19.000000,10.5,2021-11-11,https://www.utcourts.gov/courts/juv/juvsites/3...,"['docket_number', 'users1_birthdate', 'male', ...",https://courtformsonline.org/forms/6e420f1b357...,"['*docket_number', '*users1_birthdate', 'male'...",UT,THIRD DISTRICT JUVENILE COURT WORK PROGRAM REF...
2,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,[],5.0,4.800000,12.5,2021-11-11,https://www.utcourts.gov/courts/juv/juvsites/3...,['of the probation department or by order of t...,https://courtformsonline.org/forms/2532cd2b6d3...,"['probation_department_order_court', 'special_...",UT,Page 1 of 4\n UTAH STATE DISTRICT JUVENILE COU...
3,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,[],3.0,14.666667,6.5,2021-11-11,https://www.utcourts.gov/howto/family/adoption...,"['Name', 'Address', 'City State Zip', 'Phone',...",https://courtformsonline.org/forms/f6a6814890f...,"['*users1_name', '*users1_address_line_one', '...",UT,Consent to Adoption and Waiver of \n Rights\n ...
4,Adoption Agreement,Adopting a Minor Stepchild,['GO-00-00-00-00'],1.0,19.000000,7.5,2021-11-11,https://www.utcourts.gov/howto/family/adoption...,"['Name', 'Address', 'City State Zip', 'Phone',...",https://courtformsonline.org/forms/be656a9a361...,"['*users1_name', '*users1_address_line_one', '...",UT,Adoption\n \n Agreement\n \n Approved Board of...


In [175]:
files_df[(files_df["jurisdiction"]=="MA") & pd.isnull(files_df["text"])][["id","jurisdiction","title","text"]]

,id,jurisdiction,title,text
3224,5e7716400e4867163f60c513fa29e9a9,MA,Account,NaN
3225,2f4f038d61e725f1bdb377dfe60c143a,MA,Account Without Schedules,NaN
3226,13c091b90b3ee286afef69e55591d6f9,MA,Trust Account,NaN
3227,f1ce815b4e383259c0507e93c8b0fbfb,MA,Inventory,NaN
3228,a2b60d3b9a8613ea20c6347a8914dbec,MA,Combined Inventory Without Schedules,NaN
...,...,...,...,...
3461,822bf393b369ddcaf12a2df361812592,MA,Confidential Juror Questionnaire,NaN
3473,8e3fc98c8f01fad160e4f948951639da,MA,Application for 3-Day Commitment for Mental Il...,NaN
3487,5aaef2a7c9c735861daf8e00eef29379,MA,Registered Land Affidavit,NaN
3491,c0605de26e800c4624be191680601e05,MA,Tax Lien Motion and Notice of Hearing,NaN


In [148]:
test_vec = files_df[["id","jurisdiction","title","text"]]
#test_vec = files_df[(files_df["jurisdiction"]=="MA") & pd.notnull(files_df["text"])][["id","jurisdiction","title","text"]]
test_vec

,id,jurisdiction,title,text
0,04b3a0734774c02edf8eb9056d23954aa38e96c77c3392...,UT,Community Service Worksheet Third District Juv...,COMMUNITY SERVICE WORKSHEET\n THIRD DISTRICT J...
1,6e420f1b3575cfd8ef94b71977da9e38252e3395a78439...,UT,Third District Juvenile Court Work Program Ref...,THIRD DISTRICT JUVENILE COURT WORK PROGRAM REF...
2,2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065...,UT,Utah State District Juvenile Court Probation O...,Page 1 of 4\n UTAH STATE DISTRICT JUVENILE COU...
3,f6a6814890f21c11524d5785d772272916ff95909b7dba...,UT,Adoptee's Consent to Adoption and Waiver of Ri...,Consent to Adoption and Waiver of \n Rights\n ...
4,be656a9a361db7c3532026b0bae372e704beb160fce37e...,UT,Adoption Agreement,Adoption\n \n Agreement\n \n Approved Board of...
...,...,...,...,...
10013,3649f8e39b42bd6653f77444dbac2a9f,WY,Order Granting Motion to Terminate Stalking Or...,Order Granting Motion to Terminate Page 1 of 1...
10014,82de0286f4668c01249ac6a7b854f980,WY,Order Denying Motion to Terminate Stalking Ord...,Order Denying Motion to Terminate Page 1 of 1 ...
10015,6392da792d342e0a8813d74e36e4402b,WY,Stalking Motion and Affidavit to Show Cause,Motion and Affidavit for Order . to Show Cause...
10016,b11a0c83c9adb1053662f328adb97851,WY,Order to Show Cause in Stalking Cause,Stalking Order to Show Cause Page 1 of 1 . Rev...


In [149]:
test_vec["vec"] = test_vec["text"].apply(lit.vectorize)

/var/folders/d7/2h1shmhj4ks536wzvbblxfnw0000gn/T/ipykernel_66935/662921542.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_vec["vec"] = test_vec["text"].apply(lit.vectorize)


In [150]:
test_vec

,id,jurisdiction,title,text,vec
0,04b3a0734774c02edf8eb9056d23954aa38e96c77c3392...,UT,Community Service Worksheet Third District Juv...,COMMUNITY SERVICE WORKSHEET\n THIRD DISTRICT J...,"[-0.0004915084524791351, 0.006005721204041762,..."
1,6e420f1b3575cfd8ef94b71977da9e38252e3395a78439...,UT,Third District Juvenile Court Work Program Ref...,THIRD DISTRICT JUVENILE COURT WORK PROGRAM REF...,"[0.0012265718431578156, 0.0008261604677638675,..."
2,2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065...,UT,Utah State District Juvenile Court Probation O...,Page 1 of 4\n UTAH STATE DISTRICT JUVENILE COU...,"[0.0015724380764039548, 0.0012151133366268536,..."
3,f6a6814890f21c11524d5785d772272916ff95909b7dba...,UT,Adoptee's Consent to Adoption and Waiver of Ri...,Consent to Adoption and Waiver of \n Rights\n ...,"[0.001375169653932976, 0.0015351812439174555, ..."
4,be656a9a361db7c3532026b0bae372e704beb160fce37e...,UT,Adoption Agreement,Adoption\n \n Agreement\n \n Approved Board of...,"[0.0014871446574984686, 0.00026352259358501434..."
...,...,...,...,...,...
10013,3649f8e39b42bd6653f77444dbac2a9f,WY,Order Granting Motion to Terminate Stalking Or...,Order Granting Motion to Terminate Page 1 of 1...,"[0.0003534632229791686, 0.006019837325049232, ..."
10014,82de0286f4668c01249ac6a7b854f980,WY,Order Denying Motion to Terminate Stalking Ord...,Order Denying Motion to Terminate Page 1 of 1 ...,"[0.0009471631064577058, 0.005653379564791792, ..."
10015,6392da792d342e0a8813d74e36e4402b,WY,Stalking Motion and Affidavit to Show Cause,Motion and Affidavit for Order . to Show Cause...,"[8.61050442289801e-05, 0.005379825409675688, -..."
10016,b11a0c83c9adb1053662f328adb97851,WY,Order to Show Cause in Stalking Cause,Stalking Order to Show Cause Page 1 of 1 . Rev...,"[-0.0008141878619805573, 0.007079068276932599,..."


In [158]:
from sklearn.decomposition import PCA
import numpy as np
import re
import json

In [151]:
vec_df_low =test_vec["vec"].to_list()
pca2 = PCA(n_components=100)
pca2.fit(vec_df_low)
vec_df_low = pca2.transform(vec_df_low)
test_vec["PCA"] = list(vec_df_low)
test_vec

/var/folders/d7/2h1shmhj4ks536wzvbblxfnw0000gn/T/ipykernel_66935/1091315983.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_vec["PCA"] = list(vec_df_low)


,id,jurisdiction,title,text,vec,PCA
0,04b3a0734774c02edf8eb9056d23954aa38e96c77c3392...,UT,Community Service Worksheet Third District Juv...,COMMUNITY SERVICE WORKSHEET\n THIRD DISTRICT J...,"[-0.0004915084524791351, 0.006005721204041762,...","[-0.03282647697617556, 0.0032804589187352966, ..."
1,6e420f1b3575cfd8ef94b71977da9e38252e3395a78439...,UT,Third District Juvenile Court Work Program Ref...,THIRD DISTRICT JUVENILE COURT WORK PROGRAM REF...,"[0.0012265718431578156, 0.0008261604677638675,...","[0.06159067396389223, 0.03698548704444769, -0...."
2,2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065...,UT,Utah State District Juvenile Court Probation O...,Page 1 of 4\n UTAH STATE DISTRICT JUVENILE COU...,"[0.0015724380764039548, 0.0012151133366268536,...","[0.04582395854446053, 0.03394607915557763, -0...."
3,f6a6814890f21c11524d5785d772272916ff95909b7dba...,UT,Adoptee's Consent to Adoption and Waiver of Ri...,Consent to Adoption and Waiver of \n Rights\n ...,"[0.001375169653932976, 0.0015351812439174555, ...","[0.04711718638777453, 0.03379385792779632, -0...."
4,be656a9a361db7c3532026b0bae372e704beb160fce37e...,UT,Adoption Agreement,Adoption\n \n Agreement\n \n Approved Board of...,"[0.0014871446574984686, 0.00026352259358501434...","[0.07239486418836429, 0.03982014199992252, -0...."
...,...,...,...,...,...,...
10013,3649f8e39b42bd6653f77444dbac2a9f,WY,Order Granting Motion to Terminate Stalking Or...,Order Granting Motion to Terminate Page 1 of 1...,"[0.0003534632229791686, 0.006019837325049232, ...","[-0.013009937508010248, -0.0036086067030847956..."
10014,82de0286f4668c01249ac6a7b854f980,WY,Order Denying Motion to Terminate Stalking Ord...,Order Denying Motion to Terminate Page 1 of 1 ...,"[0.0009471631064577058, 0.005653379564791792, ...","[-0.010868624785772423, -0.006073858322225243,..."
10015,6392da792d342e0a8813d74e36e4402b,WY,Stalking Motion and Affidavit to Show Cause,Motion and Affidavit for Order . to Show Cause...,"[8.61050442289801e-05, 0.005379825409675688, -...","[-0.004760086916357266, -0.01086155962346259, ..."
10016,b11a0c83c9adb1053662f328adb97851,WY,Order to Show Cause in Stalking Cause,Stalking Order to Show Cause Page 1 of 1 . Rev...,"[-0.0008141878619805573, 0.007079068276932599,...","[-0.021810805106076366, 0.0010006600767101706,..."


In [166]:
formsinfo = "var FormsInfo = [\n\t"
i = 1
for index,row in test_vec.iterrows():
    formsinfo += "{"
    formsinfo += """
\t"fid":"%s",
\t"jur":"%s",
\t"name":%s,
\t"vec":%s
"""%(row["id"],row["jurisdiction"],json.dumps(row["title"]),np.array2string(row["PCA"],separator=','))
    formsinfo += "\t}"
    if i < len(test_vec):
        formsinfo += ","
    else:
        formsinfo += "\n"
    i+=1

formsinfo = re.sub("\n|\t","",formsinfo)
    
formsinfo += "]\n"

In [167]:
text_file = open("docs/js/formsinfo.js", "w")
text_file.write(formsinfo)
text_file.close()

In [159]:
json.dumps("this is a test'ing")

'"this is a test\'ing"'

In [9]:
def header(title,description,path="./",wide=0):
    
    if wide==1:
        pclass = "content_wide"
    else:
        pclass = "content"
    
    html = """<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<HTML xmlns="http://www.w3.org/1999/xhtml"
      xmlns:og="http://ogp.me/ns#"
      xmlns:fb="http://www.facebook.com/2008/fbml">
<HEAD>
    <title>%s</title>
    <!-- Global site tag (gtag.js) - Google Analytics -->
    <script async src="https://www.googletagmanager.com/gtag/js?id=UA-108858221-1"></script>
    <script>
      window.dataLayer = window.dataLayer || [];
      function gtag(){dataLayer.push(arguments);}
      gtag('js', new Date());

      gtag('config', 'UA-108858221-1');
    </script>
    <meta http-equiv="Content-type" content="text/html;charset=UTF-8"/>
    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" />
    <meta name="apple-mobile-web-app-capable" content="no" />
    <link rel="apple-touch-icon" href="%simages/icon_300.png"/>
    <meta property="og:type" content="website"/>
    <meta property="og:title" content="%s"/>
    <meta property="og:description" content="%s"/>
    <meta property="og:image" content="%simages/bigdata.png"/>

    <meta name="twitter:card" content="summary_large_image">
    <meta name="twitter:site" content="@SuffolkLITLab">
    <meta name="twitter:creator" content="@SuffolkLITLab">
    <meta name="twitter:title" content="%s">
    <meta name="twitter:description" content="%s">
    <meta name="twitter:image" content="http://www.davidcolarusso.com/images/bigdata.png"/>

    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <link rel="shortcut icon" type="image/x-icon" href="/favicon.ico">
    <link rel="apple-touch-icon" href="%simages/icon.png"/>
    <link rel="stylesheet" href="https://code.jquery.com/ui/1.11.1/themes/smoothness/jquery-ui.css">
    <link rel="stylesheet" type="text/css" href="%scss/style.css?v=%s">
    <script src="https://code.jquery.com/jquery-1.11.1.min.js"></script>
    <script src="https://code.jquery.com/jquery-1.10.2.js"></script>
    <script src="https://code.jquery.com/ui/1.11.1/jquery-ui.js"></script>

</HEAD>
<BODY BGCOLOR="#ffffff" BACKGROUND="" MARGINWIDTH="0" MARGINHEIGHT="0">
<div style="box-sizing: border-box;width:100%%;background:#d6392b;color:white;padding:15px;text-align:center;">
<b>This tool is in alpha</b>, and this page is just for testing purposes. <b>DO NOT RELY ON THE CONTENT FOUND HERE!</b></div>
<div class="%s">

    <div id="icon" style="background-size: 100px 100px;background-image: url('%simages/seal.jpg');"><a href="%s" alt="home"><img src="%simages/space.gif" width="100px" height="100px;" border="0" alt="LIT Logo"/></a></div>
    <h1 style="text-align:center;">
        The Legal Innovation & Technology Lab's Form Explorer <sup><font size=+1>Alpha</font></sup>
        <center style="margin-top:5px;">
          <span class="subtitle">@ Suffolk Law School&nbsp;<font style="font-weight:normal">Last updated %s</font></span>
        </center>
    </h1>
"""%(title,path,title,description,path,title,description,path,path,today,pclass,path,path,path,today)
    return html
    
def footer(path="./"):
    html = """

<div class="footer">
    <a href="/" alt="home"><img src="%simages/blue_logo.png" width="50px" align="left" border="0" alt="LIT Logo"/></a>
<font size=-1><a href="mailto:litlab@suffolk.edu">Email</a> | <a href="https://github.com/SuffolkLITLab" target="_blank">GitHub</a> | <a href="/terms">Terms &amp; Privacy</a> | <a href="/credits">Credits</a></font>
</div>

</BODY>
</HTML>
"""%(path)
    
    return html

# Home page

In [10]:
def home():
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="." class="menu" style="color:black;">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp;
      <a href="list/" class="menu">&nbsp;Lists&nbsp;</a>
      <!--<a href="sim/" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="flow/" class="menu">&nbsp;Flows&nbsp;</a>-->
      
        </p>
    </div>

  <div class="content"><h2><a name="title" href="#title" class="anchor"></a>What is the Form Explorer?</h2>

    <div style="float:left;width:100%%">
      <div class="r_img_embed" style="max-width:500px;">
        <img src="images//bigdata.png" alt="Image of the character Data as a giant standing next to the Supreme Court" width="100%%"/>
        <div class="caption">Big Data &amp; the Law, h/t <a href="https://flic.kr/p/7YZubW" target="_blank">Tim Sackton</a> &amp; <a href="https://medium.com/@wtrsld/big-data-made-me-do-it-5bfc3f46871c" target="_blank">Josh Lee</a></div>
      </div>
      <p>
        We're collecting existing pdf forms from multiple jurisdictions and making it possible for folks (e.g., courts and non-profits) to turn them into beautiful webapps with ease. Eventually, you will be able to go from form to prototype webapp in a few clicks. As time goes by, we'll grow the number of jurisdictions and improve our tools' performance. They'll never do all of the work, but they help plot a course for what otherwise might seem an "impossible" task. They'll help make things concrete, allowing for stakeholders and developers to edit their way to a production-ready solution built on suggestions rooted in years of form building experience.
      </p>
      <p>
        For many, interaction with a court involves completing a "form," and often, this means filling out either a paper form or pdf facsimile. Occasionally, the latter can be found online and come with embedded fillable fields. Most of the time, however, this just means one can type in the fields before printing the form. The infrastructure that has grown up around court forms too often focus on delivering either a physical form or pictures of such to the court. Very rare is the form that can easily be filled out on a phone absent specialty software, and even more rare is the form that helps guide users through the process. 
        There are exceptions, like the forms found at <a href="https://lawhelpinteractive.org/" target="_blank">LawHelp Interactive</a> or the growing constellation of <a href="https://www.a2jauthor.org/" target="_blank">A2J Author</a>-driven forms found across multiple jurisdictions. Yet, the majority of court processes linked to forms have failed to keep pace with our technical ability to offer context-aware mobile friendly fully-electronic interactions. This Form Explorer is part of a broader open-source effort—<a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/" target="_blank">the Document Assembly Line</a>—aimed at changing this. To learn more about how we got started, check out <a href="https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3911381" target="_blank">Digital Curb Cuts: Towards an Inclusive Open Forms Ecosystem</a>, a law review article describing the project's genesis.
      </p>
      <p>
        On this site you can explore the connections between forms and get a feel for populations of existing forms, allowing you to strategize how best to redesign or translate flat forms into vibrant interactive web apps. 
      </p>
      <h2><a name="data" href="#data" class="anchor"></a>Our Data</h2>
      <p>
        Currently, we've downloaded and parsed %s forms from %s jurisdictions. Use the menu above to explore, or download summary data here: <a href="data/form_data.csv" target="_blank">form_data.csv</a> And please remember this is a work in progress. So, everything here is subject to change including things like fields we find and how we calculate readability. 
      </p>
    </div>"""%(len(tmp_df),len(tmp_df["jurisdiction"].unique()))   
    return html

In [11]:
html = header("About the Form Explorer?","""We're collecting existing pdf forms from multiple jurisdictions and making it possible for folks (e.g., courts and non-profits) to turn them into beautiful webapps with ease. Eventually, you will be able to go from form to prototype webapp in a few clicks. As time goes by, we'll grow the number of jurisdictions and improve our tools' performance. They'll never do all of the work, but they help plot a course for what otherwise might seem an "impossible" task. They'll help make things concrete, allowing for stakeholders and developers to edit their way to a production-ready solution built on suggestions rooted in years of form building experience.""")+home()+footer()
text_file = open("docs/index.html", "w")
text_file.write(html)
text_file.close()

# redirects

In [12]:
def redirects(jur):
    html = """<html>
<head>
<META http-equiv="CACHE-CONTROL" CONTENT="NO-CACHE">
<meta http-equiv="refresh" content="0; url=%s" />
</head>
</html>
"""%jur
    return html

In [12]:
html = redirects(default_jur)

text_file = open("docs/sim/index.html", "w")
text_file.write(html)
text_file.close()

text_file = open("docs/flow/index.html", "w")
text_file.write(html)
text_file.close()

text_file = open("docs/list/index.html", "w")
text_file.write(html)
text_file.close()

# Similarity Viewer

In [13]:
def simview(jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../'+this.value">"""
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
      <a href="../../list/%s" class="menu">&nbsp;Lists&nbsp;</a>
      <a href="../../sim/%s" class="menu" style="color:black;">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>
      
        </p>
    </div>

    <div class="content_wide"><h2><a name="title" href="#title" class="anchor"></a>Similar Forms View (%s Court Forms)</h2>
        <p>
          Below you can explore the connections between forms. Each form is represented by a circle.  
          The lines between each circle represent shared data fields.
        </p>
        Choose a grouping to explore: 
        <select id="graphselect" onChange="$('#graph').attr('src', 'graphs/'+this.value+'.html');$('#grouplink').attr('href', '../../list/%s/#'+$('#graphselect').val());">
        """%(jur[0],jur[0],jur[0],jur[1],jur[0])
    
    k = 0
    for topic in files_df[files_df["jurisdiction"]==jur[0]]["group"].unique():
        html+="<option value='%s'"%(removeSpecial(topic))
        if k==0:
            html+=" SELECTED"
        html+=">%s</option>"%(topic)
        k+=1
    
    html+="""</select>
        &nbsp;&nbsp;<a href="../../list/%s/#%s" id="grouplink">visit froms in this group</span>
        <!--
            Possible library for viewer:
            https://www.sigmajs.org/
        <iframe src="https://www.sigmajs.org/demo/index.html" width="100%%" height="700px;"></iframe>
        -->
        <iframe id="graph" src="graphs/%s.html" width="100%%" height="635px;" frameBorder="0" onLoad="$('#grouplink').attr('href', '../../list/%s/#'+$('#graphselect').val());"></iframe>

    </div>
"""%(jur[0],removeSpecial(files_df[files_df["jurisdiction"]==jur[0]]["group"].unique()[0]),removeSpecial(files_df[files_df["jurisdiction"]==jur[0]]["group"].unique()[0]),jur[0])
    return html

In [14]:
for jur in jurs:    
    print(jur)
    path = "../../"
    
    if not os.path.exists("docs/sim/%s"%(jur[0])):
        os.makedirs("docs/sim/%s"%(jur[0]))
    #else:
    #    shutil.rmtree("docs/sim/%s"%(jur[0]))
    #    os.makedirs("docs/sim/%s"%(jur[0]))
        
    html = header("Similar %s Court Forms"%jur[1],"""We're collecting existing pdf forms from multiple jurisdictions and making it possible for folks (e.g., courts and non-profits) to turn them into beautiful webapps with ease. Eventually, you will be able to go from form to prototype webapp in a few clicks. As time goes by, we'll grow the number of jurisdictions and improve our tools' performance. They'll never do all of the work, but they help plot a course for what otherwise might seem an "impossible" task. They'll help make things concrete, allowing for stakeholders and developers to edit their way to a production-ready solution built on suggestions rooted in years of form building experience.""",path,wide=1)+simview(jur,path)+footer(path)
    text_file = open("docs/sim/%s/index.html"%jur[0], "w")
    text_file.write(html)
    text_file.close()

['AK', 'Alaska']
['AL', 'Alabama']
['CA', 'California']
['CO', 'Colorado']
['CT', 'Connecticut']
['DE', 'Delaware']
['HI', 'Hawaii']
['ID', 'Idaho']
['IL', 'Illinois']
['KS', 'Kansas']
['MA', 'Massachusetts']
['ME', 'Maine']
['MD', 'Maryland']
['MI', 'Michigan']
['MN', 'Minnesota']
['NC', 'North Carolina']
['NE', 'Nebraska']
['NV', 'Nevada']
['SD', 'South Dakota']
['UT', 'Utah']
['VA', 'Virginia']
['VT', 'Vermont']
['WA', 'Washington']
['WI', 'Wisconsin']


IndexError: index 0 is out of bounds for axis 0 with size 0

# Flow View

In [15]:
def flowview(jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../'+this.value">"""
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
      <a href="../../list/%s" class="menu">&nbsp;Lists&nbsp;</a>
      <a href="../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../flow/%s" class="menu" style="color:black;">&nbsp;Flows&nbsp;</a>
        </p>
    </div>


      <div class="content_wide"><h2><a name="title" href="#title" class="anchor"></a>User Flow (%s Court Forms)</h2>
        <p style="background:#fcf19d">
        <i><b>Under Construction:</b> With historical form data we could provde insights on a users's flow through forms 
          (e.g., after using form X, 20%% of people used form Y). 
          If you work with the court and can help provide historic form data, <a href="mailto:litlab@suffolk.edu">let us know</a>.
      </i></p>
        <div style="overflow-x: scroll;:hidden;overflow-y: hidden;">
          <img src="../../images/.jpg" alt="" width="1500px"/>
        </div>

      </div>
"""%(jur[0],jur[0],jur[0],jur[1])
    return html

In [16]:
for jur in jurs:    
    print(jur)
    path = "../../"
    
    if not os.path.exists("docs/flow/%s"%(jur[0])):
        os.makedirs("docs/flow/%s"%(jur[0]))
    else:
        shutil.rmtree("docs/flow/%s"%(jur[0]))
        os.makedirs("docs/flow/%s"%(jur[0]))
        
    html = header("Flows Through %s Court Forms"%jur[1],"""We're collecting existing pdf forms from multiple jurisdictions and making it possible for folks (e.g., courts and non-profits) to turn them into beautiful webapps with ease. Eventually, you will be able to go from form to prototype webapp in a few clicks. As time goes by, we'll grow the number of jurisdictions and improve our tools' performance. They'll never do all of the work, but they help plot a course for what otherwise might seem an "impossible" task. They'll help make things concrete, allowing for stakeholders and developers to edit their way to a production-ready solution built on suggestions rooted in years of form building experience.""",path,wide=1)+flowview(jur,path)+footer(path)
    text_file = open("docs/flow/%s/index.html"%jur[0], "w")
    text_file.write(html)
    text_file.close()

['AK', 'Alaska']
['AL', 'Alabama']
['CA', 'California']
['CO', 'Colorado']
['CT', 'Connecticut']
['DE', 'Delaware']
['HI', 'Hawaii']
['ID', 'Idaho']
['IL', 'Illinois']
['KS', 'Kansas']
['MA', 'Massachusetts']
['ME', 'Maine']
['MD', 'Maryland']
['MI', 'Michigan']
['MN', 'Minnesota']
['NC', 'North Carolina']
['NE', 'Nebraska']
['NV', 'Nevada']
['SD', 'South Dakota']
['UT', 'Utah']
['VA', 'Virginia']
['VT', 'Vermont']
['WA', 'Washington']
['WI', 'Wisconsin']
['WY', 'Wyoming']


# List VIew

In [17]:
def listview(jur,path="./"):
    
    pages = files_df[(files_df["jurisdiction"]==jur[0])]["pages"].mean()
    f_per_p = files_df[(files_df["jurisdiction"]==jur[0])]["f_per_p"].mean()
    fields = files_df[(files_df["jurisdiction"]==jur[0])]["f_per_p"].mean()*files_df[(files_df["jurisdiction"]==jur[0])]["pages"].mean()
    reading = files_df[(files_df["jurisdiction"]==jur[0])]["reading"].median()
    forms= len(files_df[(files_df["jurisdiction"]==jur[0])])
    
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../'+this.value">"""
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
      <a href="../../list/%s" class="menu" style="color:black;">&nbsp;Lists&nbsp;</a>
      <!--<a href="../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>-->
        </p>
    </div>
"""%(jur[0],jur[0],jur[0])
    
    html += """
    <div class="content">
    <div class="main_col">
    <h2><a name="title" href="#title" class="anchor"></a>List of %s Court Forms</h2>

    <p>
    Below you will find %s forms. The average form is %.0f pages long, with %.0f fields per page. The median form is written at a %.2f-grade reading level.
    </p>
    
    <p>Download a .csv file including meta data on all %s forms, including source urls, field names, and reading levels: 
    <a href="../../data/%s_form_data.csv" target="_blank">%s_form_data.csv</a>
    </p>
    
    Choose Ordering: 
    <select id="form_ordering" onChange="sort_links()">
        <option value='court_grouping'>Court Groupings</option>
        <option value='LIST_grouping'>LIST Groupings</option>
        <option value='most_used'>Most Used</option>
    </select>
    <script>
        function sort_links(){
            if($('#form_ordering').val() == "court_grouping"){
                $('#court_grouping').show()
                $('#LIST_grouping').hide()
                $('#most_used').hide()
            } else if($('#form_ordering').val() == "LIST_grouping"){
                $('#court_grouping').hide()
                $('#LIST_grouping').show()
                $('#most_used').hide()
            } else if($('#form_ordering').val() == "most_used"){
                $('#court_grouping').hide()
                $('#LIST_grouping').hide()
                $('#most_used').show()
            }
        }
    
    </script>
    
    <div id="court_grouping" style="width:100;">
    
    <p><i>Links are grouped by categories found on the court's website.</i></p>
    
    """%(jur[1],forms,pages,fields,reading,forms,jur[0],jur[0])

    for cat in files_df[files_df["jurisdiction"]==jur[0]]["group"].unique():
        html += """
        <h3><a name="%s" href="#%s" class="anchor"></a>%s</h3>
        <ul>"""%(removeSpecial(cat),removeSpecial(cat),cat)

        for index,row in files_df[(files_df["jurisdiction"]==jur[0]) & (files_df["group"]==cat)].iterrows():
            #print(row[1][0])
            html += """<li><a href="../../form/%s/%s.html">%s</a></li>"""%(jur[0],row["id"],row["title"])
            
        html += "</ul><p style=\"text-align:center\"><a href=\"#title\">go to top</a></p>"
        
    html+="""</div>
    
    <div id="LIST_grouping" style="display:none;width:100;">
    
    <p><i>Links are grouped by categories from the <a href="https://taxonomy.legal/" target="_blank" class="exlink">Legal Issue Taxonomy (LIST)</a>. Note: the grouping below was done by machine. Consequently, it is incomplete.</i></p>
    """
    
    lookfor = [
                ["BE","Public Benefits"],
                ["WO","Work and Emplyment Law"],
                ["ES","Estates, Wills, and Gaurdianships"],
                ["CR","Crime and Prisons"],
                ["FA","Family"],
                ["TO","Accidents and Torts"],
                ["TR","Trafic and Cars"],
                ["HE","Health"],
                ["MO","Money, Debt, and Consumer Issues"],
                ["HO","Housing"],
                ["VE","Veterans and Military"],
                ["IM","Immigration"],
                ["DI","Disaster Relief"],
                ["RI","Civil and Human Rights"],
                ["EN","Environmental Justice"],
                ["NA","Native American Issues and Tribal Law"],
                ["ED","Education"],
                ["BU","Small Business and IP"],
                ["GO","Government Services"],
                ["CO","Courts and Lawyers"],
              ]
    
    for item in lookfor:
        list_group = files_df[(files_df["list"].str.contains(item[0])) & (files_df["jurisdiction"]==jur[0]) & (files_df["list"]!="[]")]
        if len(list_group)>0:
            html+="<h3>%s</h3><ul>"%item[1]
            for index,row in list_group.iterrows():
                html += """<li><a href="../../form/%s/%s.html">%s</a></li>"""%(jur[0],row["id"],row["title"])
            html+="</ul>"
        else:
            html+="<h3>%s</h3><ul><li>No matches found</li></ul>"%item[1]
        
    html+="""
    </div>
    
    <div id="most_used" style="display:none;width:100;">
    
    <p><i>Links are orderd by most used.</i></p>
    
    <p style="background:#fcf19d">
        <i>
            <b>Under Construction:</b> With historical form data we could provde insights on how frequently forms are used.
            If you work with the court and can help provide historic form data, <a href="mailto:litlab@suffolk.edu">let us know</a>.
        </i>
    </p>
    
    </div>
    </div>
    """
    
    html+="""<div class="sidebar">
            <h2>Search %s Forms</h2>
            <table cellpadding="0px"><tr><td width="100%%">
            <input id="q" style="width:100%%">
            </td><td width="1%%">&nbsp;&nbsp;&nbsp;</td><td width="1%%">
            <input type="button" value="Search" onClick="open_search();">
            </td></tr></table>
            <script>
                sort_links();
            
                function open_search_(){
                    alert('Coming Soon!');
                }
                function open_search(){
                    window.open('https://www.google.com/search?q='+$('#q').val()+'+site:suffolklitlab.org/form-explorer/form/%s')
                }
                $("#q").on('keyup', function (e) {
                    if (e.key === 'Enter' || e.keyCode === 13) {
                        open_search()
                    }
                });
            </script>
            <h2>Standard Field Names</h2>
            <p>We maintain a bank of <a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/label_variables">standard questions and field names</a>. A list of common fields is found below, click a field to see those forms using it.</p>
            <ul>
    """%(jur[1],jur[0])
        
    for field in included_fields:
        print("\t"+field)
        html += """<li><a href="fields/%s.html">%s</a></li>"""%(field,field)
        
    html+="</ul></div>"

    return html

In [18]:
for jur in jurs:    
    print(jur)
    path = "../../"
    
    if not os.path.exists("docs/list/%s"%(jur[0])):
        os.makedirs("docs/list/%s"%(jur[0]))
    #else:
    #    shutil.rmtree("docs/list/%s"%(jur[0]))
    #    os.makedirs("docs/list/%s"%(jur[0]))
        
    html = header("%s Court Forms"%jur[1],"""We're collecting existing pdf forms from multiple jurisdictions and making it possible for folks (e.g., courts and non-profits) to turn them into beautiful webapps with ease. Eventually, you will be able to go from form to prototype webapp in a few clicks. As time goes by, we'll grow the number of jurisdictions and improve our tools' performance. They'll never do all of the work, but they help plot a course for what otherwise might seem an "impossible" task. They'll help make things concrete, allowing for stakeholders and developers to edit their way to a production-ready solution built on suggestions rooted in years of form building experience.""",path,wide=0)+listview(jur,path)+footer(path)
    text_file = open("docs/list/%s/index.html"%(jur[0]), "w", encoding="utf-8")
    text_file.write(html)
    text_file.close()

['AK', 'Alaska']
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
['AL', 'Alabama']
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
['CA', 'California']
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signatu

# Field Labels

In [22]:
def fields(field_name,jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore:</span>&nbsp;&nbsp; <select style="width:100px;" onChange="window.location.href='../../'+this.value+'/fields/%s.html'">"""%field_name
    
    for item in jurs: 
        if item[0]==jur[0]:
            selected = "SELECTED"
        else:
            selected = ""
        html += "<option value=\"%s\" %s>%s</option>"%(item[0],selected,item[1])
        
    html += """
      </select>&nbsp;&nbsp;
      <a href="../../../list/%s" class="menu">&nbsp;Lists&nbsp;</a>
      <!--<a href="../../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>-->
        </p>
    </div>
"""%(jur[0],jur[0],jur[0])
    
    regex = field_name
    forms_with_var = files_df[(files_df["jurisdiction"]==jur[0]) & files_df['fields'].str.contains(regex)]

    html += """
      <div class="content">
      <div class="main_col">
      <h2><a name="title" href="#title" class="anchor"></a>%s court forms that use: <i>%s</i></h2>
      <p>Out of <a href="../">%s forms</a>, <span style="background:yellow"><b>the following %s forms use <i>%s</i>:</b></span>
      
      To learn more about our standard field names, read <a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/label_variables">Field labels to use in template files</a>.
      
      </p>
"""%(jur[1],field_name,len(files_df[files_df["jurisdiction"]==jur[0]]),len(forms_with_var),field_name)
    
    for cat in forms_with_var["group"].unique():
        html += """
        <h3><a name="%s" href="#%s" class="anchor"></a>%s</h3>
        <ul>"""%(cat,cat,cat)

        for index,row in forms_with_var[forms_with_var["group"]==cat].iterrows():
            #print(row[1][0])
            html += """<li><a href="../../../form/%s/%s.html">%s</a></li>"""%(jur[0],row["id"],row["title"])
            
        html += "</ul><p style=\"text-align:center\"><a href=\"#title\">go to top</a></p>"
        
    html += "</div>"

    html+="""</div><div class="sidebar">
            <h2>Search %s Forms</h2>
            <table cellpadding="0px"><tr><td width="100%%">
            <input id="q" style="width:100%%">
            </td><td width="1%%">&nbsp;&nbsp;&nbsp;</td><td width="1%%">
            <input type="button" value="Search" onClick="open_search();">
            </td></tr></table>
            <script>
                function open_search_(){
                    alert('Coming Soon!');
                }
                function open_search(){
                    window.open('https://www.google.com/search?q='+$('#q').val()+'+site:suffolklitlab.org/form-explorer/form/%s')
                }
                $("#q").on('keyup', function (e) {
                    if (e.key === 'Enter' || e.keyCode === 13) {
                        open_search()
                    }
                });
            </script>
            <p style="text-align:center;background:#d2e6fa"><a href="../">BACK TO ALL %s FORMS</a></p>
            <h2>Standard Field Names</h2>
            <p>We maintain a bank of <a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/label_variables">standard questions and field names</a>. A list of common fields is found below, click a field to see those forms using it.</p>
            <ul>
    """%(jur[1],jur[0],jur[1].upper())
        
    for field in included_fields:
        print("\t"+field)
        html += """<li><a href="%s.html">%s</a></li>"""%(field,field)
        
    html+="</ul></div>"    
    
    return html

In [23]:
for jur in jurs:  
    print(jur)
    path = "../../../"
    
    if not os.path.exists("docs/list/%s/fields/"%(jur[0])):
        os.makedirs("docs/list/%s/fields/"%(jur[0]))
    
    for field in included_fields:
        print("\t"+field)
        html = header("%s Court Forms using %s"%(jur[1],field),"""We're collecting existing pdf forms from multiple jurisdictions and making it possible for folks (e.g., courts and non-profits) to turn them into beautiful webapps with ease. Eventually, you will be able to go from form to prototype webapp in a few clicks. As time goes by, we'll grow the number of jurisdictions and improve our tools' performance. They'll never do all of the work, but they help plot a course for what otherwise might seem an "impossible" task. They'll help make things concrete, allowing for stakeholders and developers to edit their way to a production-ready solution built on suggestions rooted in years of form building experience.""",path)+fields(field,jur,path)+footer(path)
        text_file = open("docs/list/%s/fields/%s.html"%(jur[0],field), "w")
        text_file.write(html)
        text_file.close()

['AK', 'Alaska']
	users1_name
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	users1_birthdate
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	users1_address_line_one
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_count

	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	users1_email
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	plantiffs1_name
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	defen

	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	users1_birthdate
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	users1_address_line_one
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature

	defendants1_name
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	petitioners1_name
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	respondents1_name
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signatu

	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	users1_address_line_two
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	users1_address_city
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signat

...

	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	docket_number
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	trial_court_county
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	u

	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	docket_number
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	trial_court_county
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	u

	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	docket_number
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	trial_court_county
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	users1_signature
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	user

	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	users1_address_city
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	users1_address_state
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature

	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	users1_signature
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
	signature_date
	users1_name
	users1_birthdate
	users1_address_line_one
	users1_address_line_two
	users1_address_city
	users1_address_state
	users1_address_zip
	users1_phone_number
	users1_email
	plantiffs1_name
	defendants1_name
	petitioners1_name
	respondents1_name
	docket_number
	trial_court_county
	users1_signature
	signature_date
['W

# Create form pages

In [28]:
def formview(jur,path="./"):
    html = """
    <div class="menu_bar">
        <p style="text-align:center;">
      <a href="https://suffolklitlab.org/" class="menu">&nbsp;LIT Lab Home&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      <a href="../../" class="menu">&nbsp;About&nbsp;The&nbsp;Explorer&nbsp;</a>&nbsp;<font style="color:#888;">|</font>&nbsp;
      &nbsp;&nbsp;<span style="font-family: sans-serif;font-size: 12px;text-transform: uppercase;">Explore: """
    
    for item in jurs: 
        if item[0]==jur[0]:
            this_state = item[1]
            html += "%s"%this_state

    html += """
      </span>&nbsp;&nbsp;
      <a href="../../list/%s" class="menu">&nbsp;Lists&nbsp;</a>
      <!--<a href="../../sim/%s" class="menu">&nbsp;Similar Forms&nbsp;</a>
      <a href="../../flow/%s" class="menu">&nbsp;Flows&nbsp;</a>-->
        </p>
    </div>

"""%(jur[0],jur[0],jur[0])
    
    if jur[0]!="CA":
        linkurl = "https://courtformsonline.org/forms/%s.pdf"%row["id"]
        iframelink = linkurl
    else:
        linkurl = "javascript:alert('Califonia machine-processed forms are not yet available.')"
        iframelink = "https://courtformsonline.org/forms/%s.pdf"%row["id"]
        
    html += """
  <div class="content_wide" style="padding-left:0px;">
    <div class="pdf">
      <iframe src="%s" width="100%%" height="1100px;"></iframe>
      <div id="form_text" style="display:none;">Here is the form text we found: %s</div>
    </div>
    <div class="pdfinfo">
      <h1><a name="title" href="#title" class="anchor"></a>%s</h1>
      <p>
        This info page is part of the LIT Lab's <i>Form Explorer</i> project. It is not associated with the %s state courts. 
        To learn more about the project, check out our <a href="../../">about page</a>.
      </p>
      <p>
        <b>Downloads:</b> In order to get the data on this page we scan forms for... You can download both the <a href="%s" target="_blank" class="exlink">original form</a> (last checked on %s) 
        and the <a href="%s">machine-processed form</a> with normalized data fields.
      </p>

        <hr style="border: 1px solid #fff;border-bottom: 1px solid #555;">
      <h3><a name="about_form" href="#about_form" class="anchor"></a>About This Form:</h3>
      <ul>"""%(iframelink,row["text"],row["title"],this_state,row["url"],row["downloaded"],linkurl)
    
    html += """
        <li>Sourced from <a href="http://%s" target="_blank" class="exlink">%s</a> on %s</li>
        <li>Court Grouping: <a href="../../list/%s/#%s">%s</a></li>
        <li>Page(s): %s </li>
        <li>Fields(s): %s </li>
        <li>Average fields per page: %s</li>
        <li>Reading Level: Grade %s</li>
        <li><a href="https://taxonomy.legal/" target="_blank" class="exlink">LIST</a> Grouping(s): 
    """%(row["source"],row["source"],row["downloaded"],jur[0],removeSpecial(row["group"]),row["group"],clean_n(row["pages"]),clean_n(row["pages"]*row["f_per_p"]),clean_n(row["f_per_p"]),clean_n(row["reading"]))
    
    if pd.notnull(row["list"]):
        j= 0
        nsmi = eval(row["list"])
        if len(nsmi)>0:
            for f in nsmi:
                html += "<a href=\"https://taxonomy.legal/term/%s\" target=\"_blank\">%s</a>"%(f,f)
                if j==len(nsmi)-1:
                    html+=". "
                else:
                    html+=", "
                j+=1    
        else:
            html+="Unknown"
    else:
            html+="Unknown"
            nsmi = []
        
    html += """        
        </li>
      </ul>
      
      <hr style="border: 1px solid #fff;border-bottom: 1px solid #555;">
      <h3><a name="about_users" href="#about_users" class="anchor"></a>About Users of This Form:<sup>&dagger;</sup></h3>
      <p style="background:#fcf19d"><sup>&dagger;</sup> <i>With historical form data we could provde insights on users of this for  
      (e.g., filers of this form file an average of <b>X</b> other forms. <b>Y%%</b> of them identify as male...).  
      If you work with the court and can help provide historic form data, <a href="mailto:litlab@suffolk.edu">let us know</a>.
      </i></p>
      
      <hr style="border: 1px solid #fff;border-bottom: 1px solid #555;">
      <h3><a name="fields" href="#fields" class="anchor"></a>Identified Data Fields and Utilization:<sup>*</sup></h3>
      <p>We have done our best to automaticly identify and name form fields according to our <a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/label_variables/">naming conventions</a>. 
      When possible, we've used names tied to our question library. See e.g., <a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/question_library/names">user1_name</a>.
      If we think we've found a match to a question in our library, it is highlighted in green. New names are auto generated. So, you will probably need to edit some of them.
      </p>
      
      <p style="background:#fcf19d"><sup>*</sup><i> With historical form data we could provde insights on field utilization  
      (e.g., <b>X%%</b> of the time this field is left blank...). 
       If you work with the court and can help provide historic form data, <a href="mailto:litlab@suffolk.edu">let us know</a>.
      </i></p>
      
      <div style="float:left;width:100%%;margin-bottom:25px;">
          <div class="tab" style="border: 1px solid #fff;border-bottom: 1px solid #555;width:15px;padding:10px 0px;">&nbsp;</div>
          <div class="tab" id="tab1" style="border-bottom: 1px solid #fff;" >
              <a href="javascript:void('');" onClick="tab_focus('orig_fields');" class="menu" id="atab1" style="color:black;">Original Order</a>
          </div>
          <div class="tab" style="border: 1px solid #fff;border-bottom: 1px solid #555;width:15px;padding:10px 0px;">&nbsp;</div>
          <div class="tab" id="tab2">
              <a href="javascript:void('');" onClick="tab_focus('sug_screens');" class="menu" id="atab2">Suggested Screens</a>
          </div>
          <div class="tab" style="border: 1px solid #fff;border-bottom: 1px solid #555;">
              &nbsp;
          </div>
      </div>
      <script>
          function tab_focus(tab){
              if (tab=="orig_fields") {
                  $('#sug_screens').hide();
                  $('#orig_fields').show();
                  $('#tab1').css('border-bottom-color', '#fff;');
                  $('#tab2').css('border-bottom-color', '#555;');
                  $('#atab1').css('color', 'black');
                  $('#atab2').css('color', '#529ecc');
              } else {
                  $('#orig_fields').hide();
                  $('#sug_screens').show();
                  $('#tab1').css('border-bottom-color', '#555;');
                  $('#tab2').css('border-bottom-color', '#fff;');
                  $('#atab1').css('color', '#529ecc');
                  $('#atab2').css('color', 'black');
              }
          }
      </script>
      
      """%()
    
    
    html+="<div id=\"orig_fields\" style=\"padding-left:15px;\"><p>Here are the fields we could identify.</p>"
    html+="<div style=\"max-height:550px;overflow-y: auto;padding-bottom:25px;\"><ul>"
        
    if not pd.isnull(row["fields"]):
        if len(eval(row["fields"]))==0:
            html+="<li>No fields found</li>"
        else:        
            j = 0
            for f in eval(row["fields"]):
                html += "<li>"
                if re.search('^\*',f):
                    f = re.sub("^\*","",f)
                    f_link = re.sub("__\d+$","",f)
                    #v = re.search('^(\w*)',f).groups()[0]
                    #f = re.sub("\|","</code></a>",f)
                    html +="<code style=\"background:#a2e874\"><a href=\"../../list/%s/fields/%s.html\">%s</a></code> was <i>%s</i> (%.2f conf)<!--100%% completed. <a href=\"\">Learn more about distribution of answers.</a>--></li>"%(jur[0],f_link,f,eval(row["fields_old"])[j],eval(row["fields_conf"])[j])   
                else: 
                    #v = re.search('^(\w*)',f).groups()[0]
                    #f = re.sub("\|","</code>",f)
                    html +="<code>%s</code> was <i>%s</i> (%.2f conf)<!--100%% completed. <a href=\"\">Learn more about distribution of answers.</a>--></li>"%(f,eval(row["fields_old"])[j],eval(row["fields_conf"])[j])
                j+=1    
                
    else:
        html+="<li>No fields found</li>"

    html+="</ul></div></div>"
        
    html+="<div id=\"sug_screens\" style=\"display:none;padding-left:15px;\"><p>We've done our best to group similar variables togther to avoid overwhelming the user.</p>"
    html+="<div style=\"max-height:550px;overflow-y: auto;padding-bottom:25px;\">"

    if not pd.isnull(row["fields"]):
        if len(eval(row["fields"]))==0:
            html+="<ul><li>No fields found</li>"
        else:       
            screens = lit.cluster_screens(eval(row["fields"]),damping=0.5)
            i = 0
            for screen in screens:
                html+="<p>Suggested Screen %s:</p><ul>"%i
                j = 0
                for f in screens[screen]:
                    html += "<li>"
                    if re.search('^\*',f):
                        f = re.sub("^\*","",f)
                        f_link = re.sub("__\d+$","",f)
                        #v = re.search('^(\w*)',f).groups()[0]
                        #f = re.sub("\|","</code></a>",f)
                        html +="<code style=\"background:#a2e874\"><a href=\"../../list/%s/fields/%s.html\">%s</a></code></li>"%(jur[0],f_link,f)   
                    else: 
                        #v = re.search('^(\w*)',f).groups()[0]
                        #f = re.sub("\|","</code>",f)
                        html +="<code>%s</code></li>"%(f)
                    j+=1    
                i+=1
                html+="</ul>"
    else:
        html+="<li>No fields found</li>"    

    html+="</div></div>"

        
    html += """
        <hr style="border: 1px solid #fff;border-bottom: 1px solid #555;">
        <!--<li><a href="../form/">show more</a></li>-->
        

      </ul>"""
    
    sim_forms = files_sim_df[files_sim_df["id_1"]==row["id"]].sort_values(by=['similarity'], ascending=False)[:5]
    
    html+="""      <h3><a name="similar" href="#similar" class="anchor"></a>Similar Forms:</h3>
      <p>For more context, visit the <a href="../../sim/%s">Similar Forms View</a>.</p>
      <ul>"""%jur[0]

    for index_2,row_2 in sim_forms.iterrows():
        html+="<li><a href=\"%s.html\">%s</a> (shares %s%% of data fields)</li>"%(row_2["id_2"],row_2["title_2"],clean_n(row_2["similarity"]*100))

    if len(sim_forms)==0:
        html+="<li>No matches found</li>"
        
    html+="""</ul>
      <!--
      <li><a href="../form/">show more</a></li>-->"""
        
    html += """
      
      <hr style="border: 1px solid #fff;border-bottom: 1px solid #555;">
      <h3><a name="before" href="#before" class="anchor"></a>Before using this form, users used:<sup>&dagger;&dagger;</sup></h3>
      <p>For more context, visit the <a href="../../flow/%s">Flows View</a>.</p>
       <p style="background:#fcf19d"><sup>&dagger;&dagger;</sup><i> With historical form data we could provde insights on 
       what forms a user used before this one. 
       If you work with the court and can help provide historic form data, <a href="mailto:litlab@suffolk.edu">let us know</a>.
      </i></p>
      
      <hr style="border: 1px solid #fff;border-bottom: 1px solid #555;">
      <h3><a name="after" href="#after" class="anchor"></a>After using this form, users used:<sup>**</sup></h3>
      <p>For more context, visit the <a href="../../flow/%s">Flows View</a>.</p>
      <p style="background:#fcf19d"><sup>**</sup><i> With historical form data we could provde insights on 
      what forms a user used after this one. 
      If you work with the court and can help provide historic form data, <a href="mailto:litlab@suffolk.edu">let us know</a>.
      </i></p>
      
      <hr style="border: 1px solid #fff;border-bottom: 1px solid #555;">
      <h3><a name="linter" href="#linter" class="anchor"></a>See Suggestions for Improving this Form:</h3>
      <p>Using the Document Assembly Line Linter you can...</p>
      <p>
        <a href="javascript:alert('Coming Soon!');">Run this form through the Linter</a>
      </p>
      
      <hr style="border: 1px solid #fff;border-bottom: 1px solid #555;">
      <h3><a name="weaver" href="#weaver" class="anchor"></a>Create an Interactive Version of this Form:</h3>
      <p>
      The Weaver creates a draft guided interview from a template form, like the one provided here. You can use the link below to open this form in the Weaver.
      To learn more, read <a href="https://suffolklitlab.org/docassemble-AssemblyLine-documentation/docs/generating_code/">"Weaving" your form into a draft interview</a>.
      </p>"""%(jur[0],jur[0])
    
    if jur[0]!="CA":
        
        linkline = "https://apps-dev.suffolklitlab.org/start/ALWeaver/assembly_line?new_session=1&form_to_use=https://courtformsonline.org/forms/%s.pdf"%row["id"] + "&title=" + urllib.parse.quote_plus(row["title"].strip()) + "&jur=%s"%jur[0] + "&nsmi=" + str(nsmi)
        
        html+="""
      <p>
        <a href="%s" target="_blank" class="launch"><div class="launch">Launch Form in Weaver</div></a>
      </p>
      """%(linkline)
    else:
        html+="""
      <p>
        <a href="javascript:alert('Not available for California Forms at this time.')" class="launch"><div class="launch">Launch Form in Weaver</div></a>
      </p>
      """
            
    html+="""
    </div>
  </div>
    """
    return html

In [29]:
# run one sample
path = "../../"
for index,row in files_df[files_df["id"]=="9f441bfaca115775970cf03550e0c78e"].iterrows():
    print(row["jurisdiction"],row["title"])
    
    html = header(row["jurisdiction"]+": "+row["title"],"""We're collecting existing pdf forms from multiple jurisdictions and making it possible for folks (e.g., courts and non-profits) to turn them into beautiful webapps with ease. Eventually, you will be able to go from form to prototype webapp in a few clicks. As time goes by, we'll grow the number of jurisdictions and improve our tools' performance. They'll never do all of the work, but they help plot a course for what otherwise might seem an "impossible" task. They'll help make things concrete, allowing for stakeholders and developers to edit their way to a production-ready solution built on suggestions rooted in years of form building experience.""",path,wide=1)+formview([row["jurisdiction"],""],path)+footer(path)
    text_file = open("docs/form/%s/%s.html"%(row["jurisdiction"],row["id"]), "w", encoding="utf-8")
    
    text_file.write(html)
    text_file.close()

CA Declaration Confirming Parentage in Stepparent Adoption


In [ ]:
path = "../../"

if 1==1:
    for jur in jurs:    
        if not os.path.exists("docs/form/%s"%(jur[0])):
            os.makedirs("docs/form/%s"%(jur[0]))
        else:
            shutil.rmtree("docs/form/%s"%(jur[0]))
            os.makedirs("docs/form/%s"%(jur[0]))
        

for index,row in files_df.iterrows():
    print(row["jurisdiction"],row["title"])
    
    html = header(row["jurisdiction"]+": "+row["title"],"description goes here",path,wide=1)+formview([row["jurisdiction"],""],path)+footer(path)
    text_file = open("docs/form/%s/%s.html"%(row["jurisdiction"],row["id"]), "w", encoding="utf-8")
    
    text_file.write(html)
    text_file.close()

In [23]:
files_sim_df[files_sim_df["similarity"]!=0]["similarity"].median()

0.0606060606060606